# The frequency domain

We have already talked briefly about convolutions. Turns out there is another way to do them. To see why this would be important, let us consider the time it takes to do one convolution. Suppose we covolve an image (M * M) with a kernel (N * N), the time it takes to do this convolution is O(M^2*N^2). For sufficiently large N, this can be very slow. Before we look at the other method, it is helpful to understand two ways of representing images:
<br><br>
**Spatial Domain**
This is how we've been representing images. An image is represented by its pixel values. In this domain we can manipulate images either through point operations, where we can set the value of each individual pixel independent of its neighbors or through a filter, like we have been doing with convolution [1](https://www.dynamsoft.com/blog/insights/image-processing/image-processing-101-image-enhancement/).
<br><br>
**Frequency Domain**
This is another way to look at images. In this domain, we analyze images according to their frequencies.

In [ ]:
#We will read-in and convert an image from the spatial domain
# to the frequency domain.
# We will use OpenCV's dct function to do this as the Fourier
# transform helps us view an image in the frequency domain.

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
img = cv2.imread('images/bluebird.jpg', cv2.IMREAD_GRAYSCALE)
plt.imshow(img)

In [ ]:
bird_dft = np.fft.fft2(img)

In [ ]:
bird_dft_plot = 20 * np.log(np.abs(bird_dft))

In [ ]:
plt.imshow(bird_dft_plot)

We now shift to the middle to see the centre.

In [ ]:
bird_dft_shift = np.fft.fftshift(bird_dft)

In [ ]:
bird_dft_shift_plot = 20 * np.log(np.abs(bird_dft_shift))

In [ ]:
plt.imshow(bird_dft_shift_plot)

In [ ]:
print(bird_dft_shift[0, 0], bird_dft_shift[0, 473])
print(bird_dft_shift[354, 0], bird_dft_shift[354, 473])

In [ ]:
print(bird_dft[0, 0], bird_dft[0, 473])
print(bird_dft[354, 0], bird_dft[354, 473])

In [ ]:
np.fft.fft2??

In [ ]:
Ts = 1/50
t = np.arange(0, 10, Ts)
x = np.sin(2 * np.pi * 15 * t) + np.sin(2 * np.pi * 20 * t) #sin(2 pi frequency time)
plt.plot(t, x)

In [ ]:
y = np.fft.fft(x)
fs = 1/Ts
f = np.arange(0, len(y)) * fs/len(y) #k(index)/N(no of elements in fft) * R(sampling rate)

In [ ]:
plt.plot(f, abs(y)) # abs(y) == magnitude == sqrt(re**2 + img**2)

Another Frequency With Noise

In [ ]:
xnoise = np.random.randn(len(y))
x = x + xnoise
plt.plot(t, x)

In [ ]:
y_dirty = np.fft.fft(x)
y_dirty_shift = np.fft.fftshift(y_dirty)
fshift = np.arange(-len(x)/2, len(x)/2) * ((1/Ts) / len(x))
plt.plot(fshift, abs(y_dirty_shift))

In [ ]:
y_clean = y_dirty.copy()
y_clean[abs(y_clean) < 100] = 0
x_clean = np.fft.ifft(y_clean)
plt.plot(t, x_clean)

## Now what of an image

We will first plot the magnitude of the sample image in a cartesian plot.

In [ ]:
#read an image
python = cv2.imread('images/python.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
python_fft = np.fft.fft2(python)

`np.fft.fft2(python)` is equal to `np.fft.fft(np.fft.fft(python, axis=1), axis=0)`. A 2D fft is similar to a 1D fft along the column and another 1D fft on the result along the rows.

In [ ]:
#let us plot the data
python_fft_shift = np.fft.fftshift(python_fft)
plt.imshow(np.log(abs(python_fft_shift) + 1), cmap='viridis')

Now to plot the values in a grid

In [ ]:
# Create a new figure
plt.figure()

# Plot each complex number as an arrow
for num in python_fft_shift.flat:
    plt.arrow(0, 0, num.real, num.imag, head_width=0.1, head_length=0.2, fc='blue', ec='blue')

# Set the limits of the plot
plt.xlim(-4000, 4000)
plt.ylim(-4300, 4300)

# Add labels and a grid
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.grid(True)

# Show the plot
plt.show()

## Changing the magnitude and phase

We are going to change the magnitude and phase of the image above and see what it does to the image. We will take an image, plot the polar plot like the one above. We will plot another polar plot with a random magnitude and another with random phase. We will then plot the images to see how they came out.

In [ ]:
def plot_polar_image(image):
    """
    Given any image, we will produce the Argand diagram
    of the image.

    This function expects that 'image' is of complex dtype
    and is the result of a dft

    We will also find the ifft and plot as an image.
    """

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12, 4)

    # Plot each complex number as an arrow
    for num in image.flat:
        ax1.arrow(0, 0, num.real, num.imag, head_width=0.1,
                  head_length=0.2, fc='blue', ec='blue')

    
    # Set the limits of the plot
    ax1.set_xlim(-5000, 5000);
    ax1.set_ylim(-5000, 5000);
    ax1.set_xlabel('Re');
    ax1.set_ylabel('Im');
    ax1.grid(True)

    image_ifft = np.fft.ifft2(image)
    
    ax2.imshow(abs(image_ifft))

Let us try with the first image.

In [ ]:
plot_polar_image(python_fft)

Let us create a function changer for our image.

In [ ]:
def change_image(image, new_part, is_mag=0):
    """
    This function takes an image, changes the image's
    magnitude/phase and displays the image.

    The shapes of image and new_part must be equal
    Image must be a complex - result of an fft

    image - a complex array, a result of fft
    new_part - either new magnitude or phase with which
        to change the image into
    is_mag - if new_part is magnitude, set to 0,
    else if new_part is phase, set to non-zero

    Calls plot_polar_image on result
    """

    #Split the current image into magnitude and phase
    magnitude = abs(image)
    phase = np.angle(image)

    new_image = np.empty(image.shape, np.complex128)
    if is_mag == 0:
        #This means we are to swap magnitude
        #Make a complex number Data[...,0] + 1j * Data[...,1]
        new_image = (new_part * np.cos(phase)) + 1j * (new_part * np.sin(phase))
    else:
        #Swapping phase
        new_image = (magnitude * np.cos(new_part)) + 1j * (magnitude * np.sin(new_part))

    plot_polar_image(new_image)        

In [ ]:
magnitude = abs(python_fft_shift)
magnitude_change = magnitude * 1.5 * np.random.rand(magnitude.shape[0], magnitude.shape[1])

In [ ]:
change_image(python_fft, magnitude_change)

In [ ]:
magnitude_change